In [1]:
import cv2
import numpy as np 
import HandTrackingModule as htm
import time
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [13]:
wCam,hCam=640,480
cap=cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)




True

In [ ]:
pTime=0
cTime=0
detector=htm.handDetector(detectionCon=0.7)
#connect to the device system!
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0
while True:
    succes,img=cap.read()
    img=detector.findHands(img)
    lmList,boxx=detector.findPosition(img)
    if len(lmList)!=0:
        # we will work in finger point  4 and point 8 
        x1,y1=lmList[4][1],lmList[4][2]
        x2,y2=lmList[8][1],lmList[8][2]
        #make circle in point 4 and point 8 
        cv2.circle(img,(x1,y1),15,(255,0,255),cv2.FILLED)
        cv2.circle(img,(x2,y2),15,(255,0,255),cv2.FILLED)
        #make a ligne between 4 and 8 
        cv2.line(img,(x1,y1),(x2,y2),(255,0,255),3)
        #getting the center of this line ! and make a circle 
        cx,cy=(x1+x2)//2,(y1+y2)//2
        cv2.circle(img,(cx,cy),15,(255,0,255),cv2.FILLED)
        #length of this line betwen 4 and 8 
        length=math.hypot(x2-x1,y2-y1)
        #print(int(length))
        #our hand range 50-300
        #Volume range is from -65 to 0 
        vol=np.interp(length,[50,230],[minVol,maxVol])
        volBar=np.interp(length,[50,230],[400,150])
        volPer=np.interp(length,[50,230],[0,100])
        volume.SetMasterVolumeLevel(vol,None)
        #change color if length is small !
        if length<50:
                cv2.circle(img,(cx,cy),15,(0,255,0),cv2.FILLED)
        cv2.rectangle(img,(50,150),(85,400),(255,0,0),3)
        cv2.rectangle(img,(50,int(volBar)),(85,400),(255,0,0),cv2.FILLED)
        cv2.putText(img,str(int(volPer)),(40,450),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),5)



    cTime=time.time()
    fps=1/(cTime-pTime) 
    pTime=cTime
    #show  fps (10,70) for the position , 3 mean the skale , the 1 for letter ( big , small )
    cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,255),1)
    cv2.imshow("Image",img) 

    cv2.waitKey(1)